In [ ]:
from genericpath import exists
import os
import zipfile

SOURCE3_DIR = '/content/SOURCE3/'
os.makedirs('/content/SOURCE3/', exist_ok=True)
data_path = '/content/drive/MyDrive/Datasets/SOURCE3/curated.zip'
splits_path = '/content/drive/MyDrive/Datasets/SOURCE3/split.zip'

with zipfile.ZipFile(data_path, 'r') as arc_f:
  arc_f.extractall(SOURCE3_DIR)

with zipfile.ZipFile(splits_path, 'r') as arc_f:
  arc_f.extractall(SOURCE3_DIR)

SOURCE1_DIR = '/content/SOURCE1/'
os.makedirs(SOURCE1_DIR, exist_ok=True)
archive_path = '/content/drive/MyDrive/Datasets/SOURCE1/by_class.zip'
with zipfile.ZipFile(archive_path, 'r') as arc_f:
  arc_f.extractall(SOURCE1_DIR)

In [ ]:
SOURCE3_CLASSES = os.listdir("/content/SOURCE3/curated")
ascii_source3_intersection = [i for i in range(128) if str(i) in SOURCE3_CLASSES]

SOURCE3_CLASSES_MAPPINGS = {i: chr(i) for i in ascii_source3_intersection}
SPECIAL_PLACEMENT_BOTTOM = [',', '.', '_']
SPECIAL_PLACEMENT_TOP = ['"', "*",  "'", '^', '`']

SOURCE3_CLASSES_COUNTS = {SOURCE3_CLASSES_MAPPINGS[c]: len(os.listdir(os.path.join("/content/SOURCE3/curated"))) for c in ascii_source3_intersection}


{'!': 188, '"': 188, '#': 188, '$': 188, '%': 188, '&': 188, "'": 188, '(': 188, ')': 188, '*': 188, '+': 188, ',': 188, '-': 188, '.': 188, '/': 188, '0': 188, '1': 188, '2': 188, '3': 188, '4': 188, '5': 188, '6': 188, '7': 188, '8': 188, '9': 188, ':': 188, ';': 188, '<': 188, '=': 188, '>': 188, '?': 188, '@': 188, 'A': 188, 'B': 188, 'C': 188, 'D': 188, 'E': 188, 'F': 188, 'G': 188, 'H': 188, 'I': 188, 'J': 188, 'K': 188, 'L': 188, 'M': 188, 'N': 188, 'O': 188, 'P': 188, 'Q': 188, 'R': 188, 'S': 188, 'T': 188, 'U': 188, 'V': 188, 'W': 188, 'X': 188, 'Y': 188, 'Z': 188, '[': 188, ']': 188, '^': 188, '_': 188, '`': 188, 'a': 188, 'b': 188, 'c': 188, 'd': 188, 'e': 188, 'f': 188, 'g': 188, 'h': 188, 'i': 188, 'j': 188, 'k': 188, 'l': 188, 'm': 188, 'n': 188, 'o': 188, 'p': 188, 'q': 188, 'r': 188, 's': 188, 't': 188, 'u': 188, 'v': 188, 'w': 188, 'x': 188, 'y': 188, 'z': 188, '{': 188, '|': 188, '}': 188, '~': 188}


In [ ]:
!pip install english-words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 44.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for english-words: filename=english_words-2.0.1-py3-none-any.whl size=8196236 sha256=729fb26568d49ee3680839efb02c71523efd7e7efe3ce3159c695105610b0b74
  Stored in directory: /root/.cache/pip/wheels/f0/e6/d9/16a123647999fe535f03a36e7af23eef203736d84c7ca25b0b
Successfully built english-words


In [ ]:
import pandas as pd

freq_df = pd.read_csv('/content/drive/MyDrive/Datasets/etc/unigram_freq.csv')
freq_df = freq_df.sort_values(by=['count'], axis=0, ascending=False)
freq_words = freq_df['word'].to_numpy()
freq_df.head()

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


In [ ]:
from english_words import get_english_words_set
import numpy as np

words_list = list(get_english_words_set(['web2']))

SOURCE2_WORDS_PATH = '/content/drive/MyDrive/Datasets/SOURCE2/words_new.txt'
words_txt = []
with open(SOURCE2_WORDS_PATH, 'r') as w_f:
    lines = w_f.readlines()
    for line in lines:
        if line[0] == "#":
            continue
        spl = line.strip().split(" ")
        words_txt.append(spl[-1])

unique_source2_words = np.unique(np.array(words_txt))
freq_without_source2 = [w for w in freq_words if w not in unique_source2_words]
freq_source2_intersection = [w for w in freq_words if w in unique_source2_words]


In [ ]:
model_vocab_size = 50250

freq_new_size = model_vocab_size - len(unique_source2_words)
cropped_freq = freq_without_source2[:freq_new_size]

unique_source2_words, source2_counts = np.unique(words_txt, return_counts=True)
sorted_pairs = sorted(zip(unique_source2_words, source2_counts), key=lambda x: x[1], reverse=True)
sorted_unique, sorted_counts = zip(*sorted_pairs)

sorted_unique = list(sorted_unique)
sorted_counts = list(sorted_counts)

total_counts = sum(sorted_counts)
probabilities = [count / total_counts for count in sorted_counts]

In [ ]:
from PIL import Image
import numpy as np
import cv2

def get_grayscale_image(filepath):
  img = Image.open(filepath).convert('L')
  return np.array(img)

def crop_grayscale(img, inv=False):
  if inv:
    img = cv2.bitwise_not(img)
  _, bin_img = cv2.threshold(img, 128, 255, type=cv2.THRESH_BINARY_INV)
  contours, _ = cv2.findContours(bin_img, mode=cv2.RETR_EXTERNAL,
                              method=cv2.CHAIN_APPROX_SIMPLE)

  if not contours:
    return img

  y_min, y_max, x_min, x_max = np.inf, -np.inf, np.inf, -np.inf

  for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    y_min = min(y, y_min)
    y_max = max(y+h, y_max)
    x_min = min(x, x_min)
    x_max = max(x+w, x_max)

  croppped_img = bin_img[y_min:y_max, x_min:x_max]

  return croppped_img

In [50]:
import os
import random
from PIL import Image, ImageOps
import time
import shutil
import numpy as np

train_dir = "/content/SOURCE1/by_class/train"
source3_train_dir = '/content/SOURCE3/curated'
SOURCE3_CLASSES_MAPPINGS = {chr(i): i for i in ascii_source3_intersection}

classes_imgs = {}
classes_lengths = {}

for c in os.listdir(train_dir):
  classes_imgs[c] = os.listdir(os.path.join(train_dir, c))
  classes_lengths[c] = len(classes_imgs[c])

source3_classes_imgs = {}
source3_classes= [d for d in os.listdir(source3_train_dir) if os.path.isdir(os.path.join(source3_train_dir, str(d)))]
source3_only_classes = [chr(int(i)) for i in source3_classes if chr(int(i)) not in [*classes_imgs]]

for c in source3_classes:
  source3_classes_imgs[c] = [f for f in os.listdir(os.path.join(source3_train_dir, c)) if '._' not in f]

def generate_word(word, output_path):

    char_imgs = []
    word = list(word.strip())
    max_height = 0
    widths = []
    for c in word:

        if c in source3_only_classes:
          class_dir = os.path.join(source3_train_dir, str(SOURCE3_CLASSES_MAPPINGS[c]))
          class_len = len(source3_classes_imgs[str(SOURCE3_CLASSES_MAPPINGS[c])])

          idx = random.randint(0, class_len - 1)
          img = get_grayscale_image(os.path.join(
              class_dir, source3_classes_imgs[str(SOURCE3_CLASSES_MAPPINGS[c])][idx]))

          char = Image.fromarray(crop_grayscale(img, inv=True), mode='L').convert('RGB')

        else:
          class_dir = os.path.join(train_dir, c)
          idx = random.randint(0, classes_lengths[c] - 1)
          img = get_grayscale_image(os.path.join(class_dir, classes_imgs[c][idx]))

          char = Image.fromarray(crop_grayscale(img), mode='L').convert('RGB')

        max_height = max(max_height, char.size[1])
        widths.append(char.size[0])
        char_imgs.append(char)

    padding_range = (0, 4)
    padded_chars = []
    random_paddings = [random.randint(*padding_range) for i in range(len(char_imgs) - 1)] + [0]

    for char_idx in range(len(char_imgs)):
        char_img = char_imgs[char_idx]
        vert_padding = (max_height - char_img.size[1]) // 2 + 3

        horz_padding = random_paddings[char_idx]
        if word[char_idx] in SPECIAL_PLACEMENT_BOTTOM:
          vert_padding = (max_height - char_img.size[1]) + 6
          pad_img = ImageOps.expand(ImageOps.invert(char_img), (4, vert_padding, horz_padding, 0), (255, 255, 255))
        elif word[char_idx] in SPECIAL_PLACEMENT_TOP:
          vert_padding = (max_height - char_img.size[1]) + 6
          pad_img = ImageOps.expand(ImageOps.invert(char_img), (4, 0, horz_padding, vert_padding), (255, 255, 255))
        else:
          pad_img = ImageOps.expand(ImageOps.invert(char_img), (0, vert_padding, horz_padding, vert_padding), (255, 255, 255))

        padded_chars.append(pad_img)

    new_im = Image.new('RGB', (sum(widths) + sum(random_paddings), max_height + 6), color=(255, 255, 255))
    x_offset = 0
    for im in padded_chars:
      new_im.paste(im, (x_offset, 0))
      x_offset += im.size[0]
    new_im.save(output_path)


def generate_filename(frmt='jpg'):
    return str(time.time()) + '_' + str(random.randint(100, 999)) + '.' + frmt


In [55]:
import shutil

for j in range(1, 9):
  out_dir = os.path.join('/content/words/train/', "batch_" + str(j))
  filenames_list = []
  words_list = []
  os.makedirs(out_dir, exist_ok=True)

  for idx in range(50000):

    if random.random() < len(sorted_unique) / model_vocab_size:
        word = np.random.choice(sorted_unique, p=probabilities)
    else:
        word = random.choice(cropped_freq)

    word = str(word)

    filename = generate_filename()
    while os.path.exists(os.path.join(out_dir, filename)):
      filename = generate_filename()

    filenames_list.append(filename)
    words_list.append(word)

    generate_word(word, os.path.join(out_dir, filename))

  batch_df = pd.DataFrame({
      'filename': filenames_list,
      'transcription': words_list
  })

  # Save DataFrame to CSV
  batch_df.to_csv(os.path.join('/content/words/train/', "batch_" + str(j) + '.csv'), index=False)

  shutil.make_archive("/content/words_" + "batch_" + str(j), "zip", out_dir)